In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Data Preprocessing
def load_pneumonia_data():
    # Assuming X-ray images are loaded as `images` and labels `labels` (0: normal, 1: pneumonia)
    images = np.array(...)  # Load X-ray images
    labels = np.array(...)  # Load labels

    # Normalize pixel values to [0, 1]
    images = images.astype('float32') / 255.0

    # Resize to 224x224 for compatibility with InceptionV3
    images = tf.image.resize(images, (224, 224))

    # Split the data into training, validation, and test sets
    X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test

# Step 2: Data Augmentation
def augment_pneumonia_data(X_train):
    datagen = ImageDataGenerator(
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
    )
    return datagen.flow(X_train, batch_size=32)

# Step 3: Model Development using Transfer Learning (InceptionV3)
def build_pneumonia_model():
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False  # Freeze layers for transfer learning

    # Add custom layers for binary classification
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=x)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

    return model

# Step 4: Training the model
X_train, X_test, y_train, y_test = load_pneumonia_data()
train_generator = augment_pneumonia_data(X_train)

model = build_pneumonia_model()
history = model.fit(train_generator, epochs=10, validation_data=(X_test, y_test))

# Step 5: Evaluation
# Plot training & validation accuracy and loss
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.title('Model Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.title('Model Loss')
plt.legend()
plt.show()

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")  

In [ ]:
valid_indices = ~np.isnan(labels)  # Remove NaN labels
images, labels = images[valid_indices], labels[valid_indices]


In [ ]:
from tensorflow.keras.utils import to_categorical
labels = to_categorical(labels, num_classes=2)


In [ ]:
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = datagen.flow(X_train, y_train, subset='training', batch_size=32)
val_generator = datagen.flow(X_train, y_train, subset='validation', batch_size=32)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3)


In [ ]:
history = model.fit(train_generator, epochs=20, validation_data=val_generator, callbacks=[early_stopping, lr_scheduler])


In [ ]:
base_model.trainable = True
for layer in base_model.layers[:-30]:  # Keep initial layers frozen
    layer.trainable = False

# Compile again with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.00001), loss='binary_crossentropy', metrics=['accuracy'])
history_fine_tune = model.fit(train_generator, validation_data=val_generator, epochs=10)


In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = dict(enumerate(class_weights))

history = model.fit(train_generator, epochs=10, validation_data=val_generator, class_weight=class_weights)


In [ ]:
from sklearn.metrics import confusion_matrix, roc_curve, auc
import seaborn as sns

# Confusion Matrix
y_pred = (model.predict(X_test) > 0.5).astype(int)
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Normal', 'Pneumonia'], yticklabels=['Normal', 'Pneumonia'])
plt.title('Confusion Matrix')
plt.show()

# ROC Curve
y_pred_prob = model.predict(X_test).ravel()
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)
plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.2f}')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend()
plt.show()


In [ ]:
model.save('pneumonia_detection_model.h5')

# Reload for later use
from tensorflow.keras.models import load_model
model = load_model('pneumonia_detection_model.h5')


In [ ]:
import cv2
import numpy as np

def grad_cam(model, img, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(last_conv_layer_name).output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    conv_outputs = conv_outputs @ pooled_grads[..., tf.newaxis]
    conv_outputs = tf.nn.relu(conv_outputs)

    heatmap = tf.reduce_mean(conv_outputs, axis=-1)
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap.numpy()
